# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [31]:
%pip install pymongo

from pymongo import MongoClient
import pandas as pd
import re 

from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


In [32]:

from datetime import datetime

In [33]:
cliente=MongoClient()
db=cliente.ufo

In [34]:
cliente.list_database_names() 

['admin', 'companies', 'config', 'local', 'ufo']

In [35]:
!pip install folium
import folium

In [36]:
cliente=MongoClient()

db=cliente.ufo
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ufo')

In [37]:
data=pd.read_csv(r'ufo.csv')
data.head()

,Unnamed: 0,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,2004,1242.667772
1,1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,2005,1325.486319
2,2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,2008,6515.416577
3,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,2004,1211.971352
4,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,2004,6960.923396


In [38]:
ufo=data.copy()

ufo=ufo[(ufo.country=="us")]


ufo


,Unnamed: 0,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,2004,1242.667772
3,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,2004,1211.971352
4,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,2004,6960.923396
5,5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,2007,427.334113
7,7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,1999,1295.827792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78504,80326,9/9/2013 21:00,woodstock,ga,us,sphere,20.0,20 seconds,Driving 575 at 21:00 hrs saw a white and green...,9/30/2013,34.101389,-84.519444,2013,318.719195
78505,80328,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,2013,2620.021539
78506,80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,2013,3126.991417
78507,80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,2013,904.740389


In [39]:

#ufo.datetime=pd.to_datetime(ufo1.datetime,format='%Y-%m-%d %H:%M')

In [40]:

ufo=ufo.drop(['shape','state','duration','total_time','Unnamed: 0','date_posted','comments','year'], axis=1)
ufo

,datetime,city,country,latitude,longitude,distance
0,10/10/1949 20:30,san marcos,us,29.883056,-97.941111,1242.667772
3,10/10/1956 21:00,edna,us,28.978333,-96.645833,1211.971352
4,10/10/1960 20:00,kaneohe,us,21.418056,-157.803611,6960.923396
5,10/10/1961 19:00,bristol,us,36.595000,-82.188889,427.334113
7,10/10/1965 23:45,norwalk,us,41.117500,-73.408333,1295.827792
...,...,...,...,...,...,...
78504,9/9/2013 21:00,woodstock,us,34.101389,-84.519444,318.719195
78505,9/9/2013 22:00,boise,us,43.613611,-116.202500,2620.021539
78506,9/9/2013 22:00,napa,us,38.297222,-122.284444,3126.991417
78507,9/9/2013 22:20,vienna,us,38.901111,-77.265556,904.740389


In [41]:
ufo.reset_index(inplace=True)

In [42]:

#ufo_index=ufo_index.drop(['level_0','index'], axis=1)
#ufo['datetime'] = pd.DatetimeIndex(ufo['datetime']).year

ufo.head()

,index,datetime,city,country,latitude,longitude,distance
0,0,10/10/1949 20:30,san marcos,us,29.883056,-97.941111,1242.667772
1,3,10/10/1956 21:00,edna,us,28.978333,-96.645833,1211.971352
2,4,10/10/1960 20:00,kaneohe,us,21.418056,-157.803611,6960.923396
3,5,10/10/1961 19:00,bristol,us,36.595000,-82.188889,427.334113
4,7,10/10/1965 23:45,norwalk,us,41.117500,-73.408333,1295.827792


In [43]:
def data():
    ufo.insert(2, 'geo_loc', pd.Series(list(range(len(ufo)))))
    ufo['geo_loc']=ufo.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return ufo

In [44]:
ufo=data()
print('numero total de compañias consideras:', len(ufo))

numero total de compañias consideras: 63399


In [45]:
db.geoloc.insert_many(ufo.to_dict('records'))

In [46]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [69]:
ufo["year"]=ufo["datetime"].str.extract(r'(19\d{2}|20\d{2})').astype(int)

C:\Users\alima\AppData\Local\Temp/ipykernel_11188/2040148026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ufo["year"]=ufo["datetime"].str.extract(r'(19\d{2}|20\d{2})').astype(int)


In [71]:
#ufo.datetime=ufo.datetime.astype(int)
ufo = ufo[(ufo.distance > 290) & (ufo.distance < 310)]
ufo.reset_index(drop = True, inplace = True)

ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      228 non-null    int64  
 1   datetime   228 non-null    object 
 2   geo_loc    228 non-null    object 
 3   city       228 non-null    object 
 4   country    228 non-null    object 
 5   latitude   228 non-null    float64
 6   longitude  228 non-null    float64
 7   distance   228 non-null    float64
 8   year       228 non-null    int32  
dtypes: float64(3), int32(1), int64(1), object(4)
memory usage: 15.3+ KB


In [25]:
def geoquery(ufo):
    

    
    for i in range(len(ufo)):
        
        q=list(
            db.geoloc.find(
                {'geo_loc': {'$near':{'$geometry':{
                    'type':'Point',
                    'coordinates':[ufo.longitude[i], ufo.latitude[i]]
                },
                                     '$maxDistance':310}}}
            )
        )
        
    return ufo

In [26]:
def geomap(ufo, zoom_start=1.5):

    ufo.index=range(len(ufo)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(ufo)):

        folium.CircleMarker([float(ufo.latitude[i]), float(ufo.longitude[i])], popup=str(ufo.city[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    mapa.save('images/geomap.html')
    
    
    return mapa
            

In [27]:
ufo=geoquery(ufo)
ufo.head()

,index,datetime,geo_loc,city,country,latitude,longitude,distance
0,12,10/10/1970 19:00,"[-83.7619444, 37.1536111]",manchester,us,37.153611,-83.761944,302.476051
1,151,10/10/2006 12:37,"[-83.9583333, 34.8761111]",blairsville,us,34.876111,-83.958333,305.862395
2,877,10/1/2009 18:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767
3,964,10/12/1973 19:00,"[-86.8025, 33.5205556]",birmingham (just south of),us,33.520556,-86.802500,295.975266
4,1192,10/13/1969 22:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767


In [28]:
mapa=geomap(ufo.loc[:228])
mapa

In [29]:
Tiene toda la pinta de que es Nashville

SyntaxError: invalid syntax (Temp/ipykernel_11188/3310667142.py, line 1)